In [3]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.8.0-cp314-cp314-win_amd64.whl.metadata (11 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.8.0-cp314-cp314-win_amd64.whl (8.1 MB)
   ---------------------------------------- 0.0/37.3 MB ? eta -:--:--
   - -------------------------------------- 1.3/37.3 MB 6.5 MB/s eta 0:00:06
   -- ------------------------------------- 2.1/37.3 MB 4.9 MB/s eta 0:00:08
   -- ------------------------------------- 2.4/37.3 MB 3.7 MB/s eta 0:00:10
   --- ------------------------------------ 2.9/37.3 MB 3.3 MB/s eta 0:00:11
   --- ------------------------------------ 3.4/37.3 MB 3.2 MB/s eta 0:00:11
   ---- ----------------------------------- 3.9/37.3 MB 3.2 MB/s eta 0:00:11
   ---- ----------------------------------- 4.2/37.3 MB 2.9 MB/s eta 0:00:12
   ---- ----------------------------------- 4.5/37.3 MB 2.8 MB/s eta 0:00:12
   ----- --------------

In [1]:
import numpy as np
import pandas as pd
import os
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [201]:
path='C:\\Users\\User\\Documents\\Zomato Restaurant Project'
dataset=pd.read_csv(os.path.join(path,'zomato.csv'),encoding='latin-1')

In [202]:
dataset.info()

<class 'pandas.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   str    
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   str    
 4   Address               9551 non-null   str    
 5   Locality              9551 non-null   str    
 6   Locality Verbose      9551 non-null   str    
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   str    
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   str    
 12  Has Table booking     9551 non-null   str    
 13  Has Online delivery   9551 non-null   str    
 14  Is delivering now     9551 non-null   str    
 15  Switch to order menu  9551 non-n

In [204]:
country_df=pd.read_excel(os.path.join(path,'Country-Code.xlsx'))

In [219]:
final_dataset=dataset.merge(country_df,on='Country Code',how='left')
columns=['Country Code','Locality Verbose','Is delivering now','Switch to order menu','Rating color','Currency']
final_dataset.drop(columns=columns,inplace=True)
final_dataset=final_dataset[final_dataset['Country']=='India']

In [206]:
final_dataset.columns

Index(['Restaurant ID', 'Restaurant Name', 'City', 'Address', 'Locality',
       'Longitude', 'Latitude', 'Cuisines', 'Average Cost for two',
       'Has Table booking', 'Has Online delivery', 'Price range',
       'Aggregate rating', 'Rating text', 'Votes', 'Country'],
      dtype='str')

In [221]:
column=['Restaurant Name','Cuisines','Locality','City','Has Table booking','Has Online delivery','Average Cost for two','Aggregate rating']
final_dataset[column]

,Restaurant Name,Cuisines,Locality,City,Has Table booking,Has Online delivery,Average Cost for two,Aggregate rating
624,Jahanpanah,"North Indian, Mughlai",Agra Cantt,Agra,No,No,850,3.9
625,Rangrezz Restaurant,"North Indian, Mughlai",Agra Cantt,Agra,No,No,700,3.5
626,Time2Eat - Mama Chicken,North Indian,Agra Cantt,Agra,No,No,500,3.6
627,Chokho Jeeman Marwari Jain Bhojanalya,Rajasthani,Civil Lines,Agra,No,No,400,4.0
628,Pinch Of Spice,"North Indian, Chinese, Mughlai",Civil Lines,Agra,No,No,1000,4.2
...,...,...,...,...,...,...,...,...
9271,D Cabana,"Continental, Seafood, Chinese, North Indian, B...",Sagar Nagar,Vizag,No,No,600,3.6
9272,Kaloreez,"Cafe, North Indian, Chinese",Siripuram,Vizag,No,No,400,3.7
9273,Plot 17,"Burger, Pizza, Biryani",Siripuram,Vizag,No,No,600,4.3
9274,Vista - The Park,"American, North Indian, Thai, Continental","The Park, Lawsons Bay",Vizag,No,No,1500,3.8


In [88]:
final_dataset['Cuisines']=final_dataset['Cuisines'].str.split(",").apply(lambda x:[i.strip() for i in x])
final_dataset['Cuisines']=final_dataset['Cuisines'].apply(lambda x: [i.replace(" ","_") for i in x])
final_dataset['Cuisines']=final_dataset['Cuisines'].apply(lambda x : ' '.join(x))

In [98]:
final_dataset.Cuisines.sample(5)

2987      North_Indian Street_Food Mithai
5261                     Bakery Fast_Food
6526                                 Cafe
6469    Chinese North_Indian South_Indian
7366                                 Cafe
Name: Cuisines, dtype: str

<h1> Cases:</h1>
<ol>
    <li>show top 10 similar restaurants to the one user selected based on cuisines filter by city.
    <li>conditions:locality is given by user,top 10 similar restaurants in locality by cuisines.
    <li>conditions: minimum rating given
        <ul>
            <li>price range is not given-->sort them from primium to cheap</li>
            <li>price range is given-->filter by price range and then sort them in desc</li>
        </ul></li>
    <li>conditions: when ratings,price range,table booking,online delivery -->re rank them based on condition</li>
</ol>

In [101]:
tfidf= TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,1),
    lowercase=True
)
tfidf_matrix=tfidf.fit_transform(final_dataset['Cuisines'])
similarity_matrix=cosine_similarity(tfidf_matrix,tfidf_matrix)

In [190]:
restaurant_index=pd.Series(final_dataset.index-624,index=final_dataset['Restaurant Name'])

def recommendation_func(restaurant_name,restaurant_index,n):
    idx=restaurant_index[restaurant_name]
    scores=list(enumerate(similarity_matrix[idx]))
    sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
    sim_scores=sorted_scores[1:n]
    restaurant_indices=[i[0] for i in sim_scores]
    recommend=final_dataset.iloc[restaurant_indices].copy()
    recommend['similarity_score']=[i[1] for i in sim_scores]
    return recommend

In [182]:
final_dataset.loc[final_dataset['Restaurant Name']=="Subway"].tail()

,Restaurant ID,Restaurant Name,City,Address,Locality,Longitude,Latitude,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Price range,Aggregate rating,Rating text,Votes,Country
8736,3707,Subway,Noida,"B-1/24, Central Market, Near City Centre Metro...",Sector 50,77.361918,28.570513,American Fast_Food Salad Healthy_Food,500,No,No,2,2.5,Average,165,India
8884,308758,Subway,Noida,"A-44-45, Galaxy IT Park, Sector 62, Noida",Sector 62,77.370417,28.624539,American Fast_Food Salad Healthy_Food,500,No,Yes,2,2.5,Average,28,India
8923,7909,Subway,Noida,"C Block, Logic Cyber Park, Sector 62, Noida",Sector 62,77.366493,28.612903,American Fast_Food Salad Healthy_Food,500,No,Yes,2,2.4,Poor,88,India
9041,4482,Subway,Noida,"Food Court, 2nd Floor, Spice World Mall, Secto...","Spice World Mall, Sector 25",77.339801,28.586405,American Fast_Food Salad Healthy_Food,500,No,Yes,2,3.4,Average,93,India
9070,18070483,Subway,Noida,"Shop 3, Supertech Shopprix Mall, Sector 61, Noida","Supertech Shopprix Mall, Sector 61",77.364981,28.596898,American Fast_Food Salad Healthy_Food,500,No,Yes,2,2.3,Poor,24,India


In [191]:
recommendation=recommendation_func("Subway",restaurant_index,5)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [187]:
recommendation.head()

,Restaurant ID,Restaurant Name,City,Address,Locality,Longitude,Latitude,Cuisines,Average Cost for two,Has Table booking,Has Online delivery,Price range,Aggregate rating,Rating text,Votes,Country,similarity_score
1240,313149,Smaaash,Gurgaon,"Cyber Hub, DLF Cyber City, Gurgaon","Cyber Hub, DLF Cyber City",77.088913,28.495005,North_Indian Thai Continental,1400,No,No,3,3.9,Good,622,India,0.797645
8074,18303688,Smaaash,Noida,"4th Floor, DLF Mall of India, Sector 18, Noida","DLF Mall of India, Sector 18, Noida",77.320824,28.567330,North_Indian Thai Continental,1350,No,No,3,4.0,Very Good,121,India,0.797645
6769,18382353,DEL - Roseate House,New Delhi,"Roseate House, Asset 10, Hospitality District,...","Roseate House, Aerocity",77.120533,28.550802,Continental American North_Indian,3000,Yes,No,4,3.6,Good,30,India,0.788404
738,18385443,Koramangala Social,Bangalore,"118, Koramangala Industrial Area, Koramangala ...",Koramangala 7th Block,77.614130,12.935662,Continental American,1500,No,No,3,4.5,Excellent,1288,India,0.753570


In [188]:
recommendation2=recommendation("Subway",restaurant_index,5)
recommendation2.head()

TypeError: 'DataFrame' object is not callable

In [ ]:
def name_to_id(restaurant_name):
    if restaurant_name not in dataset:
        return f"Restaurant is not available."
    elif (dataset['restaurant_name']==restaurant_name) & (dataset['has_branch']==1)]:
        call a function for showing all the restaurant with name == restaurant_name
    else:
        id=dataset[dataset['restaurant_name']==restaurant_name]['restaurant id']
    return id

In [200]:
def showing_restaurant(restaurant_name):
    return dataset[dataset['restaurant name']==restaurant_name]

In [ ]:
class Recommendation_System():
    def __init__(self):
        self.sim_matrix=None
        self.dataset=None
        self.tfidf_matrix=None
        self.index=None
        
    def load_data(self,file_path):
        dataset_path=os.path.join(file_path,'zomato.csv')
        df=pd.read_csv(dataset_path,encoding='latin-1')
        df['Cuisines']=df['Cuisines'].str.split(',').apply(lambda x: [val.trim() for val in x])
        df['Cuisines']=df['Cuisines'].apply(lambda x:[val.replace(" ","_") for val in x])
        df['Cuisines']=df['Cuisines'].apply(lambda x: " ".join(x))
        self.dataset=merged_df
        print(f'Successfully loaded and process the dataset.')
        return self.dataset

    def model_develop(self):
        df=self.dataset
        tfidf=TfidfVectorizer(
            max_features=5000,
            n_grams=(1,1),
            lowercase=True
        )
        vectors=tfidf.fit_transform(df['Cuisines'])
        similarity_matrix=cosine_similarity(vectors,vectors)
        restaurant_idx={row['Restaurant ID']: i for i,row in df.iterrows()}
        self.sim_matrix=similarity_matrix
        self.index=restaurant_idx
        self.tfidf_matrix=vectors
        print(f"model successfully build.")

    def name_to_id(self,name):
        result=self.dataset[self.dataset['Restaurant Name']==name]
        if len(result)==0:
            return f'Restaurant not found.'
        else:
            columns=['Restaurant Name','Cuisines','Locality','City','Has Table booking','Has Online delivery','Average Cost for two','Aggregate rating']
            output_df=result[columns]
        return output_df

    def get_recommendation()